In [1]:
import numpy as np
import pandas as pd
import pickle

import xgboost as xgb
print(f"{xgb.__version__=}")

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.variables import wc_training_vars


xgb.__version__='3.0.1'


In [2]:
with open(f"../intermediate_files/all_df.pkl", "rb") as f:
    all_df = pickle.load(f)
print(all_df.shape)

(178827, 435)


In [3]:
with open(f"../training_outputs/first_multiclass_training/predictions.pkl", "rb") as f:
    pred_df = pickle.load(f)
print(pred_df.shape)
merged_df = pd.merge(all_df, pred_df, on=["filetype", "run", "subrun", "event"], how="left")
print(merged_df.shape)
preselected_merged_df = merged_df.query("wc_kine_reco_Enu > 0 and wc_shw_sp_n_20mev_showers > 0")
print(preselected_merged_df.shape)

(43399, 16)
(178827, 447)
(43399, 447)


In [4]:
model = xgb.XGBClassifier()
model.load_model(f"../training_outputs/first_multiclass_training/bdt.json")

x = preselected_merged_df[wc_training_vars].to_numpy()

y_from_inference = model.predict_proba(x)

y_from_inference


array([[5.56047773e-03, 1.15287339e-03, 6.13883140e-06, ...,
        1.56066617e-05, 6.62781667e-06, 1.28455600e-02],
       [2.26524798e-03, 1.80735860e-05, 1.37177540e-05, ...,
        5.73542911e-06, 1.21377176e-04, 1.28351320e-02],
       [3.86017673e-02, 1.57335610e-03, 4.77125868e-05, ...,
        3.17941085e-05, 1.55255639e-05, 7.20495760e-01],
       ...,
       [3.96875665e-03, 2.85274675e-03, 4.39947553e-06, ...,
        1.52794546e-05, 3.59801314e-04, 9.54899609e-01],
       [1.43061143e-05, 2.29366433e-05, 4.63590623e-05, ...,
        5.37958986e-04, 1.35147829e-07, 9.97808754e-01],
       [6.49400135e-06, 1.04000819e-05, 1.04882702e-07, ...,
        1.04474850e-06, 5.72910540e-06, 9.99902606e-01]], dtype=float32)

In [5]:
y_from_saved_df = preselected_merged_df[['prob_1gNp',
       'prob_1g0p', 'prob_1gNp1mu', 'prob_1g0p1mu', 'prob_1g_outFV',
       'prob_2gNp', 'prob_2g0p', 'prob_2gNp1mu', 'prob_2g0p1mu',
       'prob_2g_outFV', 'prob_other']].to_numpy()
y_from_saved_df

array([[5.56047773e-03, 1.15287339e-03, 6.13883140e-06, ...,
        1.56066617e-05, 6.62781667e-06, 1.28455600e-02],
       [2.26524798e-03, 1.80735860e-05, 1.37177540e-05, ...,
        5.73542911e-06, 1.21377176e-04, 1.28351320e-02],
       [3.86017673e-02, 1.57335610e-03, 4.77125868e-05, ...,
        3.17941085e-05, 1.55255639e-05, 7.20495760e-01],
       ...,
       [3.96875665e-03, 2.85274675e-03, 4.39947553e-06, ...,
        1.52794546e-05, 3.59801314e-04, 9.54899609e-01],
       [1.43061143e-05, 2.29366433e-05, 4.63590623e-05, ...,
        5.37958986e-04, 1.35147829e-07, 9.97808754e-01],
       [6.49400135e-06, 1.04000819e-05, 1.04882702e-07, ...,
        1.04474850e-06, 5.72910540e-06, 9.99902606e-01]], dtype=float32)

In [6]:
flattened_y_from_saved_df = y_from_saved_df.reshape(-1)
flattened_y_from_inference = y_from_inference.reshape(-1)

np.allclose(flattened_y_from_saved_df, flattened_y_from_inference)

True